### **Generate the demographic table across all studies**

This notebook will simply aggregate information across all cohorts in a clean demographic table.

In [2]:
# Imports 
import os

import pandas as pd
import numpy as np

In [3]:
# Set root Dir for the project.
ROOT_DIR = "/Users/anthonygagnon/Documents/École/Université/Projects/nf-pediatric/"
RESULTS_DIR = ROOT_DIR + "DemoTable/"

# Dataset folder
PING_DATA = "/Volumes/T7/PING/TabularData/"
BCP_DATA = ROOT_DIR + "/BCP/"
BANDA_DATA = "/Volumes/T7/CCPM/BANDA/BANDARelease1.1/"
GESTE_DATA = "/Volumes/T7/CCPM/GESTE/"
ABCD_DATA = "/Volumes/T7/CCPM/ABCD/Release_5.1/abcd-data-release-5.1/"
MYRNA_DATA = "/Users/anthonygagnon/Documents/École/Université/Projects/nf-pediatric/"

# Create a directory for the results if it doesn't exist
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

In [4]:
# Load the all_priors dataframe from the Priors folder.
df = pd.read_csv(ROOT_DIR + "Priors/All_priors.csv")
df.head(5)

,subject_id,session_id,ad_1fiber,rd_1fiber,fa_1fiber,md_ventricle,sex,age,cohort
0,sub-NDARINV00CY2MDM,ses-baseline,0.001641,0.000350,0.751306,0.002951,M,10.833333,ABCD
1,sub-NDARINV059VLAHF,ses-baseline,0.001613,0.000346,0.749973,0.002810,F,9.250000,ABCD
2,sub-NDARINV08DNLREC,ses-baseline,0.001688,0.000357,0.753631,0.002876,M,10.000000,ABCD
3,sub-NDARINV08K0R9C4,ses-baseline,0.001574,0.000355,0.738919,0.002975,M,10.583333,ABCD
4,sub-NDARINV095EVLDD,ses-baseline,0.001625,0.000341,0.755531,0.002852,M,10.250000,ABCD


In [11]:
# Create two rows per cohort: one for male with the median age and range in parenthesis, and the second for female with the same info.
demo_table = []
cohorts = ["MYRNA", "BCP", "ABCD", "GESTE", "BANDA", "PING"]
for cohort in cohorts:
    cohort_df = df[df['cohort'] == cohort]

    for sex in ['M', 'F']:
        sex_df = cohort_df[cohort_df['sex'] == sex]
        n_count = len(sex_df)
        # Fetch age info
        age_info = f"{sex_df['age'].median():.2f} ({sex_df['age'].min():.1f}-{sex_df['age'].max():.1f})"
        demo_table.append({
            "Cohorts": cohort,
            "Sex": sex,
            "N": n_count,
            "Age (range) in years": age_info
        })

# Add a total row
demo_table.append({
    "Cohorts": "Total",
    "Sex": "",
    "N": df.shape[0],
    "Age (range) in years": f"{df['age'].median():.2f} ({df['age'].min():.1f}-{df['age'].max():.1f})"
})
demo_df = pd.DataFrame(demo_table)
demo_df.to_csv(RESULTS_DIR + "demographic_table.csv", index=False)

#### **Let's calculate the amount of subjects per scanner brand**

- GESTE and MYRNA are only using a Phillips scanner.
- BCP and BANDA are only using a Siemens scanner.
- PING are using a mixed GE, Siemens, and Phillips, so need to load the csv containing site info.
- ABCD are using Siemens are GE, so need to load the csv containing site info.

In [10]:
# using the all_priors dataframe, add a new column for the scanner manufacturer for each subject in the dataframe.
# For currently unknown, set nothing.
df['scanner'] = ""

# MYRNA and GESTE = Phillips
df.loc[df['cohort'].isin(['MYRNA', 'GESTE']), 'scanner'] = 'Philips'

# BCP = Siemens
df.loc[df['cohort'] == 'BCP', 'scanner'] = 'Siemens'

# BANDA = Siemens
df.loc[df['cohort'] == 'BANDA', 'scanner'] = 'Siemens'

In [11]:
# Load the image03.txt for the PING study.
scanner_PING = pd.read_csv(ROOT_DIR + "PING/image03.txt", sep="\t")
scanner_PING.drop(0, inplace=True)  # Drop the first row which contains header info

# Extract src_subject_id and scanner_manufacturer_pd
scanner_PING = scanner_PING[['src_subject_id', 'scanner_manufacturer_pd']]
scanner_PING.rename(columns={'src_subject_id': 'subject_id', 'scanner_manufacturer_pd': 'scanner'}, inplace=True)
scanner_PING['subject_id'] = "sub-" + scanner_PING['subject_id'].astype(str)
scanner_PING.head()

# Now, append the scanner info to the df for each subject in the PING cohort.
for index, row in scanner_PING.iterrows():
    subject_id = row['subject_id']
    scanner = row['scanner']
    # Add a check to validate the subject exists in the df
    if ((df['cohort'] == 'PING') & (df['subject_id'] == subject_id)).any():
        df.loc[(df['cohort'] == 'PING') & (df['subject_id'] == subject_id), 'scanner'] = scanner

In [18]:
# Load the ABCD image03.txt for the ABCD study.
scanner_ABCD = pd.read_csv(ABCD_DATA + "core/imaging/mri_y_adm_info.csv")
scanner_ABCD = scanner_ABCD[['src_subject_id', 'mri_info_manufacturer']]
scanner_ABCD.rename(columns={'src_subject_id': 'subject_id', 'mri_info_manufacturer': 'scanner'}, inplace=True)
scanner_ABCD['subject_id'] = "sub-" + scanner_ABCD['subject_id'].astype(str).str.replace('_', '')

# Now, append the scanner info to the df for each subject in the ABCD cohort.
for index, row in scanner_ABCD.iterrows():
    subject_id = row['subject_id']
    scanner = row['scanner']
    # Add a check to validate the subject exists in the df
    if ((df['cohort'] == 'ABCD') & (df['subject_id'] == subject_id)).any():
        df.loc[(df['cohort'] == 'ABCD') & (df['subject_id'] == subject_id), 'scanner'] = scanner

In [22]:
# Assess if the scanner column has any empty values.
empty_scanner = df[df['scanner'] == '']
print(f"Number of subjects with unknown scanner: {empty_scanner.shape[0]}")

# Homogeneize the scanner names
df['scanner'] = df['scanner'].replace({'SIEMENS': 'Siemens',
                                       'Philips Medical Systems': 'Philips',
                                       'GE MEDICAL SYSTEMS': 'GE'})

# Print the missing scanner subjects
print("Subjects with unknown scanner:")
print(empty_scanner[['subject_id', 'cohort']])

# Replace the empty scanner values with 'Unknown'
df['scanner'] = df['scanner'].replace({'': 'Unknown'})

# Print the counts and percentages of each scanner type in the dataset.
scanner_counts = df['scanner'].value_counts()
scanner_percentages = df['scanner'].value_counts(normalize=True) * 100
print("Scanner counts:")
print(scanner_counts)
print("\nScanner percentages:")
print(scanner_percentages)

Number of subjects with unknown scanner: 13
Subjects with unknown scanner:
     subject_id cohort
1601  sub-P1372   PING
1691  sub-P1715   PING
1692  sub-P1716   PING
1693  sub-P1744   PING
1694  sub-P1746   PING
1695  sub-P1747   PING
1696  sub-P1750   PING
1697  sub-P1751   PING
1698  sub-P1754   PING
1699  sub-P1755   PING
1700  sub-P1756   PING
1701  sub-P1757   PING
1702  sub-P1762   PING
Scanner counts:
scanner
Siemens    1021
Philips     400
GE          269
Unknown      13
Name: count, dtype: int64

Scanner percentages:
scanner
Siemens    59.953024
Philips    23.487962
GE         15.795655
Unknown     0.763359
Name: proportion, dtype: float64


In [23]:
# Save the updated dataframe with scanner info to a new CSV file.
df.to_csv(ROOT_DIR + "Priors/All_priors_with_scanner.csv", index=False)